# **Iris Flower Classification using Gradio and Ensemble Learning**

This project builds an Iris Flower Classifier with a Gradio interface for easy use. It uses multiple machine learning models in a Voting Classifier to improve accuracy..

---

## **Overview of the Project**
### **Key Components**
1. **Dataset**: Uses the **Iris dataset** containing flower measurements.
2. **Feature Extraction**: Extracts **Sepal Length, Sepal Width, Petal Length, and Petal Width** as input features.
3. **Model Selection**: Implements **mutiple ML models**.
4. **Voting Classifier**: Combines all models to make a **final prediction**.
5. **Deployment with Gradio**: Creates a **simple web UI** for predictions.

---

## **Step-by-Step Code Explanation**



### **Step 1: Import Necessary Libraries**

In [5]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [7]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


In [8]:
import gradio as gr
import pandas as pd
import pickle
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
)
from sklearn.linear_model import (
    LogisticRegression, RidgeClassifier, Perceptron, PassiveAggressiveClassifier, SGDClassifier
)
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV # Import CalibratedClassifierCV



- **Gradio**: Used for creating the user interface where users can input data and receive predictions.
- **Pandas**: To load and manipulate the dataset (iris data).
- **Pickle**: To save the trained model so that it can be reused without retraining.
- **Various Machine Learning Models**: Models like **Logistic Regression**, **Random Forest**, **XGBoost**, **SVC**, and others are imported to build the ensemble classifier.

---

### **Step 2: Load and Prepare the Dataset**

In [9]:
import seaborn as sns
iris_data = sns.load_dataset("iris")
iris_data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### **Step 3: Split Data into Training and Testing Sets**

In [10]:
# Extract features and target
X = iris_data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y = iris_data['species']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### **Step 4: Define Multiple Classifiers for the Ensemble**

- **Ensemble Learning**: This section defines a dictionary of multiple classifiers, including both basic models like **Logistic Regression** and more complex models like **Random Forest** and **XGBoost**.

In [11]:
# Define multiple classifiers
models = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'LDA': LinearDiscriminantAnalysis(),
    'Ridge Classifier': RidgeClassifier(),
    'Perceptron': Perceptron(),
    'Passive Aggressive': PassiveAggressiveClassifier(),
    'SGD Classifier': SGDClassifier(),
    'SVC': SVC(probability=True),
    'Linear SVC': CalibratedClassifierCV(LinearSVC()),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Gaussian NB': GaussianNB(),
    'Multinomial NB': MultinomialNB(),
    'MLP': MLPClassifier()
}

### **Step 5: Create the Voting Classifier**

- The **Voting Classifier** combines the predictions of all the models in the `models` dictionary.
- It uses **hard voting**, which means the final prediction is based on the majority vote of all classifiers.

---

In [12]:
# Create a Voting Classifier
voting_clf = VotingClassifier(
    estimators=[(name, model) for name, model in models.items()],
    voting='hard'  # 'soft' for probability-based voting, 'hard' for majority voting
)

### **Step 6: Train the Ensemble Model**

In [13]:
# Train the ensemble model
voting_clf.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


VotingClassifier(estimators=[('Logistic Regression',
                              LogisticRegression(max_iter=200)),
                             ('LDA', LinearDiscriminantAnalysis()),
                             ('Ridge Classifier', RidgeClassifier()),
                             ('Perceptron', Perceptron()),
                             ('Passive Aggressive',
                              PassiveAggressiveClassifier()),
                             ('SGD Classifier', SGDClassifier()),
                             ('SVC', SVC(probability=True)),
                             ('Linear SVC',
                              CalibratedClassifierCV(estimator=LinearSVC())),
                             ('KNN', KNeighborsClassifier()),
                             ('Decision Tree', DecisionTreeClassifier()),
                             ('Random Forest', RandomForestClassifier()),
                             ('Extra Trees', ExtraTreesClassifier()),
                             ('Gradient Boosting',
                              GradientBoostingClassifier()),
                             ('AdaBoost', AdaBoostClassifier()),
                             ('Gaussian NB', GaussianNB()),
                             ('Multinomial NB', MultinomialNB()),
                             ('MLP', MLPClassifier())])

### **Step 7: Save the Trained Model Using Pickle**


- The trained **Voting Classifier** model is saved to a file using **Pickle**. This allows the model to be loaded later for making predictions without needing to retrain it.

---

In [14]:
# Save the trained model using pickle
with open('iris_voting_model.pkl', 'wb') as file:
    pickle.dump(voting_clf, file)


### **Step 8: Define the Prediction Function**

- This function is used to make predictions based on user input. It loads the saved model, passes the input data through it, and returns the predicted class of the flower.
- **Pickle** is used here to reload the saved model for prediction, avoiding the need to retrain it each time.

---

In [15]:
def predict_iris_class(sepal_length, sepal_width, petal_length, petal_width):
    # Load the model
    with open('iris_voting_model.pkl', 'rb') as file:
        loaded_model = pickle.load(file)

    input_data = [[sepal_length, sepal_width, petal_length, petal_width]]
    prediction = loaded_model.predict(input_data)[0]

    # Ensure the prediction is a string (especially for NumPy or Pandas types)
    return str(prediction)

# **Gradio**
  
Gradio is a **Python library** that lets you easily create **interactive web interfaces** for machine learning models, APIs, and Python functions. It helps in **quick testing, prototyping, and deployment** without needing complex coding.

## **Key Features of Gradio**
- **Easy to Use** – Convert any Python function into a web-based UI.
- **Supports Multiple Inputs/Outputs** – Handles text, numbers, images, audio, and video.
- **Live Deployment** – Shareable public links for easy access.
- **Compatible with ML Models** – Works with TensorFlow, PyTorch, Scikit-learn, etc.

---


### **Step 9: Creating the Gradio Interface**

🔹 **How Gradio Interface is Set Up?**  
- `fn=predict_iris_class` → Uses the defined function for prediction.
- `inputs=[gr.Number(...)]` → Accepts **four numerical inputs** (sepal and petal dimensions).
- `outputs="text"` → Displays the predicted **Iris flower class** as text.
- `title` → Sets the title of the web interface.
- `description` → Provides a short explanation of the model.

---

In [16]:
# Create the Gradio interface
iface = gr.Interface(
    fn=predict_iris_class,
    inputs=[
        gr.Number(label="Sepal Length (cm)"),
        gr.Number(label="Sepal Width (cm)"),
        gr.Number(label="Petal Length (cm)"),
        gr.Number(label="Petal Width (cm)")
    ],
    outputs="text",
    title="Iris Flower Classifier with Advanced Voting Ensemble",
    description="Enter the flower's measurements to predict its class using a robust Voting Classifier with 20 algorithms."
)

### **Step 10: Launching the Interface**

- This **starts the Gradio app**.
- It provides a **local link** and a **shareable public link**.
---

In [17]:
# Launch the interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b548ef34e3fce6948a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## **Conclusion**
This script creates a **user-friendly web app** for classifying Iris flowers using a **Voting Classifier**. With **Gradio**, the model can be **easily tested and shared** without complex web development. 🚀